# Developing, Training, and Deploying a TensorFlow model on Google Cloud Platform (using CloudShell and Cloud AI Platform)

In this notebook, we will develop a Keras model to predict flight delays using TensorFlow 2.0 as the backend. Unlike [flights_model_tf2.ipynb](flights_model_tf2.ipynb), we will use bash commands so that these can be run from CloudShell. We will also deploy to Cloud AI Platform so that the model can be executed in a serverless way.

### Make sure you are using compatible pip and TensorFlow 2.0

You need pip 19.1 or higher and TensorFlow 2.0 post Aug 2019. If not, run the necessary update scripts and check the versions again.

In [ ]:
%%bash
python3 -m pip --version

In [ ]:
%%bash
python3 -c "import tensorflow as tf; print(tf.version.VERSION)"

In [ ]:
%%bash
python3 -m pip install --user --upgrade --quiet pip

In [ ]:
%%bash
python3 -m pip install --user --upgrade --quiet tf_nightly-2.0-preview

## Set environment variables

I'm doing this for the notebook. In CloudShell, you'd do:
<pre>
export PROJECT=cloud-training-demos
</pre>
etc.

In [ ]:
# change these to try this notebook out
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [ ]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## Try out different functions in the model

Reading lines

In [ ]:
%%bash
export PYTHONPATH="$PWD/flights"
python3 -m trainer.task --bucket $BUCKET --train_batch_size=3 --func=read_lines

Finding average label

In [ ]:
%%bash
export PYTHONPATH="$PWD/flights"
python3 -m trainer.task --bucket $BUCKET --num_examples=100 --func=find_average_label

Linear model

In [ ]:
%%bash
export PYTHONPATH="$PWD/flights"
gsutil -m rm -rf gs://$BUCKET/flights/trained_model
python3 -m trainer.task --bucket $BUCKET --num_examples=1000 --func=linear

gsutil ls gs://$BUCKET/flights/trained_model

Wide-and-deep model

In [ ]:
%%bash
export PYTHONPATH="$PWD/flights"
gsutil -m rm -rf gs://$BUCKET/flights/trained_model
python3 -m trainer.task --bucket $BUCKET --num_examples=1000 --func=wide_deep

gsutil ls gs://$BUCKET/flights/trained_model

In [ ]:
%%bash
model_dir=$(gsutil ls gs://$BUCKET/flights/trained_model/export | tail -1)
echo $model_dir
saved_model_cli show --tag_set serve --signature_def serving_default --dir $model_dir

## Train on full dataset using Cloud AI Platform

In this section, I use a Docker container to capture my development environment.
An alternative is to use a Python package

In [ ]:
%%writefile flights/Dockerfile
FROM gcr.io/deeplearning-platform-release/tf2-cpu
COPY trainer /flights/trainer
RUN apt update && \
    apt install --yes python3-pip && \
    pip3 install --upgrade --quiet tf-nightly-2.0-preview

ENV PYTHONPATH ${PYTHONPATH}:/babyweight_tf2
CMD ["python3", "-m", "trainer.task"]

In [ ]:
%%writefile flights/push_docker.sh
export PROJECT_ID=$(gcloud config list project --format "value(core.project)")
export IMAGE_REPO_NAME=flights_training_container
#export IMAGE_TAG=$(date +%Y%m%d_%H%M%S)
#export IMAGE_URI=gcr.io/$PROJECT_ID/$IMAGE_REPO_NAME:$IMAGE_TAG
export IMAGE_URI=gcr.io/$PROJECT_ID/$IMAGE_REPO_NAME

echo "Building  $IMAGE_URI"
docker build -f Dockerfile -t $IMAGE_URI ./
echo "Pushing $IMAGE_URI"
docker push $IMAGE_URI

<b>Note:</b> If you get a permissions/stat error when running push_docker.sh from Notebooks, do it from CloudShell:

Open CloudShell on the GCP Console

<pre>
git clone https://github.com/GoogleCloudPlatform/data-science-on-gcp
git checkout tf2
cd data-science-on-gcp/09_cloudml/flights
bash push_docker.sh
</pre>
This step takes 5-10 minutes to run

In [ ]:
%%bash
cd flights
bash push_docker.sh

In [ ]:
%%bash
JOBID=serverlessml_$(date +%Y%m%d_%H%M%S)
gsutil -m rm -rf gs://$BUCKET/flights/trained_model

#IMAGE=gcr.io/deeplearning-platform-release/tf2-cpu
IMAGE=gcr.io/$PROJECT/flights_training_container

gcloud beta ai-platform jobs submit training $JOBID \
   --staging-bucket=gs://$BUCKET  --region=$REGION \
   --master-image-uri=$IMAGE \
   --master-machine-type=n1-standard-4 --scale-tier=CUSTOM

## Deploy model

In [ ]:
%%bash
PROJECT=cloud-training-demos
BUCKET=${PROJECT}-ml
REGION=us-east1
MODEL_NAME=flights
VERSION_NAME=kfp
EXPORT_PATH=$(gsutil ls ${OUTDIR}/export | tail -1)

if [[ $(gcloud ai-platform models list --format='value(name)' | grep $MODEL_NAME) ]]; then
    echo "$MODEL_NAME already exists"
else
    # create model
    echo "Creating $MODEL_NAME"
    gcloud ai-platform models create --regions=$REGION $MODEL_NAME
fi

if [[ $(gcloud ai-platform versions list --model $MODEL_NAME --format='value(name)' | grep $VERSION_NAME) ]]; then
    echo "Deleting already existing $MODEL_NAME:$VERSION_NAME ... "
    gcloud ai-platform versions delete --model=$MODEL_NAME $VERSION_NAME
    echo "Please run this cell again if you don't see a Creating message ... "
    sleep 10
fi

# create model
echo "Creating $MODEL_NAME:$VERSION_NAME"
gcloud ai-platform versions create --model=$MODEL_NAME $VERSION_NAME --async \
       --framework=tensorflow --python-version=3.5 --runtime-version=1.14 \
       --origin=$EXPORT_PATH --staging-bucket=gs://$BUCKET

In [ ]:
!gcloud ai-platform predict --model=flights --version=kfp --json-instances=example_input.json

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License